In [ ]:
!pip install -qU datasets fsspec transformers evaluate peft gradio

In [ ]:
import gradio as gr
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

In [ ]:
# MODEL_NAME = "FacebookAI/roberta-base"
# "google/electra-base-discriminator", "FacebookAI/roberta-base", "SpanBERT/spanbert-base-cased"

spanbert = AutoModelForQuestionAnswering.from_pretrained("/kaggle/input/spanbert_full/transformers/default/1")
roberta = AutoModelForQuestionAnswering.from_pretrained("/kaggle/input/roberta_full/transformers/default/1")
electra = AutoModelForQuestionAnswering.from_pretrained("/kaggle/input/electra_full/transformers/default/1")

spanbert_lora = AutoModelForQuestionAnswering.from_pretrained("/kaggle/input/spanbert_lora/transformers/default/1")
roberta_lora = AutoModelForQuestionAnswering.from_pretrained("/kaggle/input/roberta-lora/transformers/default/1")
electra_lora = AutoModelForQuestionAnswering.from_pretrained("/kaggle/input/electra-lora/transformers/default/1")

spanbert_tok = AutoTokenizer.from_pretrained("/kaggle/input/spanbert_full/transformers/default/1")
roberta_tok = AutoTokenizer.from_pretrained("/kaggle/input/roberta_full/transformers/default/1")
electra_tok = AutoTokenizer.from_pretrained("/kaggle/input/electra_full/transformers/default/1")

spanbert_tok_lora = AutoTokenizer.from_pretrained("/kaggle/input/spanbert_lora/transformers/default/1")
roberta_tok_lora = AutoTokenizer.from_pretrained("/kaggle/input/roberta-lora/transformers/default/1")
electra_tok_lora = AutoTokenizer.from_pretrained("/kaggle/input/electra-lora/transformers/default/1")


In [ ]:
def answer_question(context, question, model_name, is_lora="No"):
    model_list = {
        "SpanBERT": spanbert,
        "RoBERTa": roberta, 
        "ELECTRA": electra,
        "SpanBERT_lora": spanbert_lora,
        "RoBERTa_lora": roberta_lora, 
        "ELECTRA_lora": electra_lora
    }

    tokenizer_list = {
        "SpanBERT": spanbert_tok,
        "RoBERTa": roberta_tok, 
        "ELECTRA": electra_tok,
        "SpanBERT_lora": spanbert_tok_lora,
        "RoBERTa_lora": roberta_tok_lora, 
        "ELECTRA_lora": electra_tok_lora
    }

    if is_lora == "Yes":
        model_name += "_lora"

    model = model_list[model_name]
    model.eval()
    tokenizer = tokenizer_list[model_name]
    
    inputs = tokenizer(question, context, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)

    answer_start_index = outputs.start_logits.argmax()
    answer_end_index = outputs.end_logits.argmax()

    predict_answer_tokens = inputs.input_ids[0, answer_start_index : answer_end_index + 1]
    return(tokenizer.decode(predict_answer_tokens))

In [ ]:
iface = gr.Interface(
    fn=answer_question,
    inputs=[
        gr.Textbox(lines=5, label="Context"),
        gr.Textbox(lines=1, label="Question"),
        gr.Dropdown(choices=["SpanBERT", "RoBERTa", "ELECTRA"], label="Choose Model:"),
        gr.Dropdown(choices=["No", "Yes"], label="LoRA model:")
    ],
    outputs="text",
    title="Question Answering Demo",
    description="Enter a context paragraph and a question. The model will return the extracted answer.",
    theme=None
)

iface.launch()